## Human PCLS: cellular crosstalk of disease associated cell states after Nintedanib treatment

In [1]:
suppressPackageStartupMessages(library(nichenetr))
suppressPackageStartupMessages(library(Seurat))
suppressPackageStartupMessages(library(tidyverse))
suppressPackageStartupMessages(library(patchwork))
suppressPackageStartupMessages(library(data.table))
suppressPackageStartupMessages(library(Matrix))
suppressPackageStartupMessages(library(igraph))
suppressPackageStartupMessages(library(gridExtra))
suppressPackageStartupMessages(library(viridis))

In [2]:
## set working directory
setwd('/home/niklas/projects/niche_environments_FIBROSIS/HUMAN_exvivo/02_figures/ASK_joint/NicheNet/')

### Path to required input data

In [3]:
dge_dir = '/home/niklas/projects/niche_environments_FIBROSIS/PCLS_human/01_data/ASK_joint/DGE_treatment_vs_FC/'
table_dir = '/home/niklas/projects/niche_environments_FIBROSIS/HUMAN_exvivo/01_data/NicheNet_inputs/'
results_dir = '/home/niklas/projects/niche_environments_FIBROSIS/HUMAN_exvivo/01_data/NicheNet_outputs/ligand_activities_FC+Nintedanib_FC_cell_circuits/'

### Load NicheNet models and networks

In [4]:
## load NicheNet models and networks ##
ligand_target_matrix <- readRDS('/home/niklas/data/nichenet_models/ligand_target_matrix_HUMAN.rds')
lr_network <- readRDS('/home/niklas/data/nichenet_models/ligand_receptor_network_HUMAN.rds')
weighted_networks_lr <- readRDS('/home/niklas/data/nichenet_models/weighted_ligand_receptor_network_HUMAN.rds')

### Function to perform NicheNet Ligand activity analysis

In [5]:
## function to perform ligand activity analysis ##
ligand_activity_analysis <- function(sender_ct, receiver_ct, geneset_oi, 
                            geneset_title,
                            pct_expr_table, pct_thresh = 0.10,
                            pearson_thresh = 0.08){
    
    
    ## retrieve genes expressed by receiver
    expr_genes_receiver = rownames(pct_expr_table[pct_expr_table[, receiver_ct] > pct_thresh, ])
    background_expr_genes = expr_genes_receiver %>% .[. %in% rownames(ligand_target_matrix)]
    
    ## retrieve genes expressed by sender
    list_expr_genes_sender = lapply(sender_ct, function(x){rownames(pct_expr_table[pct_expr_table[, x] > pct_thresh, ])})
    expr_genes_sender = list_expr_genes_sender %>% unlist() %>% unique()
    
    ## status message ##
    print(paste0("Using ", length(geneset_oi), " genes differently regulated genes in ", receiver_ct, " (",
                 geneset_title, ")"))
    
    ### STEP1: Ligand activity analysis ###
    ## Define a set of potential ligands and receptors 
    # retrieve ligands and receptors
    ligands = lr_network %>% pull(from) %>% unique()
    receptors = lr_network %>% pull(to) %>% unique()
    # ligands expressed by sender celltypes
    expr_ligands = intersect(ligands, expr_genes_sender) 
    # receptor expressed by receiver celltypes
    expr_receptors = intersect(receptors, expr_genes_receiver)
    ### status messages ###
    print(paste0("Expressed Ligands ", length(expr_ligands)))
    print(paste0("Expressed Receptors ", length(expr_receptors)))
    
    ## filter ligands
    # only consider ligands with matching receptors (according to NicheNets databases)
    potential_ligands = lr_network %>% filter(from %in% expr_ligands & to %in% expr_receptors) %>%
                        pull(from) %>% unique()
    ### status message ###
    print(paste0("Potential Ligands ", length(potential_ligands)))
    
    ## predict ligand activities
    ligand_activities = predict_ligand_activities(geneset = geneset_oi,
                                                  background_expressed_genes = background_expr_genes,
                                                  ligand_target_matrix = ligand_target_matrix,
                                                  potential_ligands = potential_ligands)

    ## rank ligands by pearson correlation coefficient
    ligand_activities = ligand_activities %>% arrange(-pearson) %>% mutate(rank = rank(desc(pearson)))
    
    # filter consider ligands with pearson's correlation >= pearson_tresh
    ligand_activities = ligand_activities %>% filter(pearson >= pearson_thresh)
    
    ### status message ###
    print(paste0("Top ranked ligands ", length(ligand_activities$test_ligand)))
    
    return(ligand_activities)  
}

In [6]:
target_gene_prediction <- function(best_upstream_ligands,geneset_oi, target_thresh = 0.33, n_targets = 500){
    
    ## identify ligand targets
    active_ligand_target_links_df = best_upstream_ligands %>% 
                                    lapply(get_weighted_ligand_target_links,geneset = geneset_oi, ligand_target_matrix = ligand_target_matrix, n = n_targets) %>% bind_rows() %>% drop_na()
    active_ligand_target_links = prepare_ligand_target_visualization(ligand_target_df = active_ligand_target_links_df, ligand_target_matrix = ligand_target_matrix, cutoff = target_thresh)
    
    ## reformat data
    order_ligands = intersect(best_upstream_ligands, colnames(active_ligand_target_links)) %>% rev() %>% make.names()
    order_targets = active_ligand_target_links_df$target %>% unique() %>% 
                    intersect(rownames(active_ligand_target_links)) %>% make.names()
    rownames(active_ligand_target_links) = rownames(active_ligand_target_links) %>% make.names() 
    colnames(active_ligand_target_links) = colnames(active_ligand_target_links) %>% make.names() # make.names() for heatmap visualization of genes like H2-T23
    
    ## final ligand-target heatmap
    vis_ligand_target = active_ligand_target_links[order_targets,order_ligands] %>% t()
    
    ## output: ligand-target matrix
    return(vis_ligand_target) 
    
}

### Run NicheNet iteratively to detect crosstalk between all cells for FC vs CC

In [7]:
## read avg expression table
avg_expr <- read.csv(paste0(table_dir, '221204_ASK_joint_Human_PCLS_FC_FC+Nintedanib_avg_expr_SCALED_cell_type_level.csv'), row.names = 1, check.names = F, header = T)
pct_expr <- read.csv(paste0(table_dir, '221204_ASK_joint_Human_PCLS_FC_FC+Nintedanib_pct_expr_cell_type_level.csv'), row.names = 1, check.names = F, header = T)

In [8]:
cell_type_names <- as.vector(colnames(avg_expr))
cell_type_labels <- c('Aberrant_Basaloid','ectopic_EC',
                      'Myofibroblasts', 'Pericytes', 'Profibrotic_Macrophages')

In [9]:
## create final results matrix
quant_results_df <- data.frame(matrix(ncol = length(cell_type_names), nrow = length(cell_type_names)))
colnames(quant_results_df) <- cell_type_names # RECEIVER (!!!)
rownames(quant_results_df) <- cell_type_names # SENDER (!!!)
quant_results_df

,Aberrant Basaloid,ectopic EC,Myofibroblasts,Pericytes,Profibrotic Macrophages
,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>
Aberrant Basaloid,NA,NA,NA,NA,NA
ectopic EC,NA,NA,NA,NA,NA
Myofibroblasts,NA,NA,NA,NA,NA
Pericytes,NA,NA,NA,NA,NA
Profibrotic Macrophages,NA,NA,NA,NA,NA


In [10]:
qual_results_df <- copy(quant_results_df)
qual_results_df

,Aberrant Basaloid,ectopic EC,Myofibroblasts,Pericytes,Profibrotic Macrophages
,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>
Aberrant Basaloid,NA,NA,NA,NA,NA
ectopic EC,NA,NA,NA,NA,NA
Myofibroblasts,NA,NA,NA,NA,NA
Pericytes,NA,NA,NA,NA,NA
Profibrotic Macrophages,NA,NA,NA,NA,NA


### Run NicheNet

In [11]:
for(i in 1:length(cell_type_names)){
    
    ## step 1: define RECEIVER and SENDER cells
    all_senders <- cell_type_names
    all_sender_labels <- cell_type_labels
    receiver_ct <- cell_type_names[i]
    receiver_labels <- cell_type_labels[i]
    
    ## step 2: define geneset of interest
    # read dge table
    dge_table <- read.csv(paste0(dge_dir, '221204_PCLS_human_ASK_joint_' , cell_type_labels[i], '_FC+Nintedanib_vs_FC_DGE_results.csv'))
    # only significant genes: pval_adj < 0.05
    dge_table = dge_table %>% filter(qval < 0.05)
    # only genes expressed in at least 10% per group
    dge_table = dge_table %>% filter(pct.FCs > 0.1)
    dge_table = dge_table %>% filter(paste0('pct.FC.Nintedanibs') > 0.1)
    # only upregulated genes with log2fc > 0.25
    dge_genes_up = dge_table %>% filter(log2fc > 0.25)
    geneset_oi <- dge_genes_up$gene
    geneset_title <- paste0(cell_type_names[i], ' (FC+Nintedanib vs. FC)')
    print(paste0(cell_type_names[i], ' (FC+Nintedanib vs. FC): Proceeding with geneset of interest of length: ', length(geneset_oi)))
    
    ## step 3: run ligand activity analysis
    top_ligands_table <- ligand_activity_analysis(all_senders, receiver_ct, geneset_oi = geneset_oi,
                                           geneset_title = geneset_title, pct_thresh = 0.10,
                                           pct_expr_table = pct_expr, pearson_thresh = 0.00)
    
    ## dissect crosstalk between all pairs of cell types quantitatively and qualitatively
    
    # for each sender cell type check which top ligands are upregulated
    for(i in 1:length(all_senders)){
        sender_ct <- all_senders[i]
        sender_label <- all_sender_labels[i]
        # read respective DGE table
        # read dge table
        dge_table <- read.csv(paste0(dge_dir, '221204_PCLS_human_ASK_joint_' , sender_label, '_FC+Nintedanib_vs_FC_DGE_results.csv'))
        
        # subset DGE table to top ligands
        ligands_dge = dge_table %>% filter(gene %in% top_ligands_table$test_ligand) 
        
        # filter DGE table by logFC and q-Value
        # if logFC > 0.25 and q-Value < 0.05, a ligand is upregulated
        ligands_dge = ligands_dge %>% filter(qval < 0.05)
        ligands_dge = ligands_dge %>% filter(log2fc > 0.25)
        
        ## add info whether gene is upregulated in SENDER cell type to ligand activity table
        top_ligands_table <- top_ligands_table %>% mutate(new_col = case_when(test_ligand %in% ligands_dge$gene ~ 1,
                                         !(test_ligand %in% ligands_dge$gene) ~ 0))
        colnames(top_ligands_table)[colnames(top_ligands_table) == 'new_col'] <- paste0(sender_label, '_UP')
        
        ### quantitative approach ### 
        # count genes in resulting DGE table --> no. of upregulated top ligands
        ct_quant_score <- length(ligands_dge$gene)
        cat('\n')
        print(paste0(sender_ct, ' --> ', receiver_ct, ' - Top ranked ligands: ', length(top_ligands_table$test_ligand)))
        print(paste0(sender_ct, ' --> ', receiver_ct, ' - Thereof upregulated: ', ct_quant_score))
        cat('\n')
        # fill final results matrix
        quant_results_df[sender_ct, receiver_ct] <- ct_quant_score
        
        ### qualitative approach ### 
        
        # filter ligands results table: upregulated ligands only
        ligands_up = top_ligands_table %>% filter(test_ligand %in% ligands_dge$gene)
        
        # filter avg expression table: only sender cell type
        ligands_avg_expr = avg_expr %>% select(sender_ct)
        # filter avg expression table: only top ligands
        ligands_avg_expr = ligands_avg_expr %>% filter(rownames(ligands_avg_expr) %in% ligands_dge$gene)
        ligands_avg_expr$gene <- rownames(ligands_avg_expr)
        rownames(ligands_avg_expr) <- NULL
        
        # merge ligands results table with ligands avg expression table
        ligands_up <- merge.data.frame(x = ligands_up, y = ligands_avg_expr, by.x = 'test_ligand', by.y = 'gene')
        
        # add new colum to ligand results table: avg expression * ligand score (pearsons rho)
        ligands_up$contribution <- ligands_up[, sender_ct] * ligands_up$pearson
        # sender celltype contribution score equals the sum of the new column
        ct_qual_score <- sum(ligands_up$contribution)
        
        # fill final results matrix
        qual_results_df[sender_ct, receiver_ct] <- ct_qual_score

    }
    
    ## save ligand activity analysis
    print(paste0('Writing ligand activity table to:  ', results_dir, 'nichenet_ligand_act_', receiver_labels, '.csv'))
    write.csv(top_ligands_table, paste0(results_dir, 'nichenet_ligand_act_', receiver_labels, '.csv'))
    
    ## save upregulated ligands + their activity analysis
    print(paste0('Writing ligand activity table (ONLY UP-regulated ligands) to:  ', results_dir, 'nichenet_ligand_act_', receiver_labels, '_UP.csv'))
    write.csv(ligands_up, paste0(results_dir, 'nichenet_ligand_act_', receiver_labels, '_UP.csv'))
    
    ## step 4: target gene prediction
    ligand_target_prediction <- target_gene_prediction(best_upstream_ligands = top_ligands_table$test_ligand, geneset_oi = geneset_oi)
    print(paste0('Writing ligand target matrix table to:  ', results_dir, 'nichenet_ligand_target_matrix_', receiver_labels, '.csv'))
    write.csv(ligand_target_prediction, paste0(results_dir, 'nichenet_ligand_target_matrix_', receiver_labels, '.csv'))
}

[1] "Aberrant Basaloid (FC+Nintedanib vs. FC): Proceeding with geneset of interest of length: 101"
[1] "Using 101 genes differently regulated genes in Aberrant Basaloid (Aberrant Basaloid (FC+Nintedanib vs. FC))"
[1] "Expressed Ligands 291"
[1] "Expressed Receptors 193"
[1] "Potential Ligands 211"
[1] "Top ranked ligands 211"

[1] "Aberrant Basaloid --> Aberrant Basaloid - Top ranked ligands: 211"
[1] "Aberrant Basaloid --> Aberrant Basaloid - Thereof upregulated: 9"



Note: Using an external vector in selections is ambiguous.
ℹ Use `all_of(sender_ct)` instead of `sender_ct` to silence this message.
ℹ See <https://tidyselect.r-lib.org/reference/faq-external-vector.html>.
This message is displayed once per session.




[1] "ectopic EC --> Aberrant Basaloid - Top ranked ligands: 211"
[1] "ectopic EC --> Aberrant Basaloid - Thereof upregulated: 44"


[1] "Myofibroblasts --> Aberrant Basaloid - Top ranked ligands: 211"
[1] "Myofibroblasts --> Aberrant Basaloid - Thereof upregulated: 71"


[1] "Pericytes --> Aberrant Basaloid - Top ranked ligands: 211"
[1] "Pericytes --> Aberrant Basaloid - Thereof upregulated: 39"


[1] "Profibrotic Macrophages --> Aberrant Basaloid - Top ranked ligands: 211"
[1] "Profibrotic Macrophages --> Aberrant Basaloid - Thereof upregulated: 28"

[1] "Writing ligand activity table to:  /home/niklas/projects/niche_environments_FIBROSIS/HUMAN_exvivo/01_data/NicheNet_outputs/ligand_activities_FC+Nintedanib_FC_cell_circuits/nichenet_ligand_act_Aberrant_Basaloid.csv"
[1] "Writing ligand activity table (ONLY UP-regulated ligands) to:  /home/niklas/projects/niche_environments_FIBROSIS/HUMAN_exvivo/01_data/NicheNet_outputs/ligand_activities_FC+Nintedanib_FC_cell_circuits/nichenet_ligand

In [12]:
write.csv(quant_results_df, paste0(results_dir, '221204_quant_crosstalk_FC+Nintedanib_FC_cell_circuit.csv'))
quant_results_df

,Aberrant Basaloid,ectopic EC,Myofibroblasts,Pericytes,Profibrotic Macrophages
,<int>,<int>,<int>,<int>,<int>
Aberrant Basaloid,9,0,7,6,9
ectopic EC,44,1,46,45,44
Myofibroblasts,71,3,76,74,65
Pericytes,39,1,40,38,33
Profibrotic Macrophages,28,2,31,30,27


In [13]:
write.csv(qual_results_df, paste0(results_dir, '221204_qual_crosstalk_FC+Nintedanib_FC_cell_circuit.csv'))
qual_results_df

,Aberrant Basaloid,ectopic EC,Myofibroblasts,Pericytes,Profibrotic Macrophages
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Aberrant Basaloid,0.1647871,0.000000000,0.07982413,0.02466443,0.156107
ectopic EC,1.3643297,0.002427122,0.65577513,0.27144307,1.341984
Myofibroblasts,2.0399994,0.017675663,1.12234574,0.41102793,1.890749
Pericytes,1.1246869,0.011990778,0.62795639,0.24885028,1.063590
Profibrotic Macrophages,1.2724791,0.037581161,0.59800577,0.23878941,1.234158
